In [ ]:
import os
import sys
import time
import glob
import warnings
from pathlib import Path


# Data Manipulation and Processing
import numpy as np
import pandas as pd
import pydicom
from PIL import Image
from wordcloud import WordCloud
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as patches

# Machine Learning & Evaluation Metrics
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve,
    auc,
    precision_recall_curve,
    average_precision_score,
    confusion_matrix,
    classification_report,
    log_loss
)

# 5. Deep Learning
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Architectures
from tensorflow.keras.applications import (
    ResNet50,
    ResNet50V2,
    DenseNet121
)

# Model Construction (Sequential, Functional, & Ensemble Layers)
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import (
    Input,
    Dense,
    GlobalAveragePooling2D,
    Dropout,
    Flatten,
    MaxPooling2D,
    Conv2D,
    BatchNormalization,
    Rescaling,
    Average,
    Concatenate
)

# Optimizers & Loss
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy

# Keras Metrics (For training monitoring)
from tensorflow.keras.metrics import (
    BinaryAccuracy,
    Precision,
    Recall,
    AUC
)

# Callbacks
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ReduceLROnPlateau,
    ModelCheckpoint
)


# 2. Load and Prepare Data

1.  **Check Directories**: First, we verify that both the original RSNA competition dataset (containing labels) and the PNG image dataset are attached to the notebook.
2.  **Load Labels**: We load the `stage_2_train_labels.csv` file.
3.  **Create Classification DataFrame**: We create a clean `df_class` DataFrame with one unique row per patient, linking each `patientId` to its corresponding `.png` filename.


In [ ]:
if not BASE_DIR.exists() :
    print(f" dataset not foundnot found.")

else:
    print(f"dataset found.")

    labels_path = BASE_DIR / 'stage_2_train_labels.csv'
    if labels_path.exists():
        df_labels = pd.read_csv(labels_path)
        print(f"\nSuccess! RSNA Labels loaded. Shape: {df_labels.shape}")



4. **Understand the Dataset Of The Labels With Basic EDA**

In [ ]:
df_labels.head()

In [ ]:
print("\n Null Value Check")
print("-" * 60)
print(df_labels.isnull().sum())

print("\n Duplicate Check")
print("-" * 60)
dup_count = df_labels.duplicated().sum()
print(df_labels.duplicated('patientId').sum())
print(f"Total duplicate rows: {dup_count}")

print("\nℹ Basic Data Information")
print("-" * 60)
df_labels.info()

print("\n Summary Statistics (Numerical Columns)")
print("-" * 60)
display(df_labels.describe())


In [ ]:
class_counts = df_labels['Target'].value_counts().sort_index()
print(df_labels['Target'].value_counts())

class_counts.plot(kind='bar', color=['teal', 'indigo'])
plt.title("Pneumonia vs Normal Cases")
plt.xlabel("Target (0=Normal, 1=Pneumonia)")
plt.ylabel("Count")
plt.xticks(rotation=0)
plt.show()

5.  **Verify Files**: We cross-reference this DataFrame with the actual files in the `PNG_DIR` to ensure we only work with images that exist, preventing "File Not Found" errors during training.

In [ ]:
df_class = df_labels.drop_duplicates('patientId')[['patientId', 'Target']].copy()
df_class['filename'] = df_class['patientId'].apply(lambda x: f"{x}.png")
df_class['Target'] = df_class['Target'].astype(str)

df_class['filepath'] = df_class['filename'].apply(lambda f: PNG_DIR / f)
df_class['file_exists'] = df_class['filepath'].apply(lambda p: p.exists())

df_class = df_class[df_class['file_exists']].copy().reset_index(drop=True)

print("Dataframe ready for EDA")


In [ ]:
df_class.head()

# 3. Exploratory Data Analysis (EDA)

With the data successfully loaded, we now explore its characteristics.




1.  **Class Distribution**: We visualize the count of "Normal" vs. "Pneumonia" cases to understand the class imbalance in the dataset.

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x='Target', data=df_class, palette='viridis')
plt.title('Distribution of Pneumonia Cases (0 = Normal, 1 = Pneumonia)', fontsize=14)
plt.xlabel('Class', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.show()

print("Class Counts:")
print(df_class['Target'].value_counts())



2.  **Metadata Extraction**: We extract valuable metadata (Age, Sex, View Position) from the original DICOM files. This step is crucial for conducting a fairness analysis later to ensure the model performs equitably across different demographic groups.

In [ ]:
print("\nExtracting metadata from DICOM")

ages, sexes, view_positions = [], [], []

start_time = time.time()

for patient_id in tqdm(df_class['patientId']):
    dcm_path = DICOM_DIR / f"{patient_id}.dcm"
    dcm_data = pydicom.dcmread(dcm_path, stop_before_pixels=True)
    ages.append(dcm_data.PatientAge)
    sexes.append(dcm_data.PatientSex)
    view_positions.append(dcm_data.ViewPosition)

df_class['Age'] = ages
df_class['Sex'] = sexes
df_class['ViewPosition'] = view_positions
df_class['Age'] = df_class['Age'].astype(int)
df_class['AgeGroup'] = pd.cut(df_class['Age'], bins=[0, 18, 40, 60, 150], labels=['Child', 'Young Adult', 'Adult', 'Senior'])

end_time = time.time()
elapsed = end_time - start_time



3.  **Metadata Distribution**: We visualize the distribution of patients by sex and age group.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
sns.countplot(x='Sex', data=df_class, ax=axes[0], palette='magma')
axes[0].set_title('Distribution by Sex')
sns.countplot(x='AgeGroup', data=df_class, ax=axes[1], palette='plasma')
axes[1].set_title('Distribution by Age Group')
plt.show()


# 4. Visualizing Image Examples

To get a better understanding of the data, let's look at some example chest X-ray images from our dataset. We will display a few samples for both **Normal** (Target = 0) and **Pneumonia** (Target = 1) cases. This helps in visually confirming the characteristics the model will learn to distinguish.

In [ ]:
def plot_image_samples(df, target, num_samples=5, figsize=(20, 5)):
    sample_df = df[df['Target'] == str(target)].sample(num_samples, random_state=42)

    fig, axes = plt.subplots(1, num_samples, figsize=figsize)
    fig.suptitle(f'Sample Images: {"Pneumonia" if target == 1 else "Normal"} Cases', fontsize=16)

    for i, (idx, row) in enumerate(sample_df.iterrows()):
        img_path = row['filepath']
        img = Image.open(img_path).convert('RGB') # Convert to RGB for consistency
        axes[i].imshow(img, cmap='gray')
        axes[i].set_title(f"Patient: {row['patientId'][:8]}...", fontsize=10)
        axes[i].axis('off')

        # If it's a pneumonia case, draw the bounding boxes
        if target == 1:
            bboxes = df_labels[df_labels['patientId'] == row['patientId']]
            for _, box_row in bboxes.iterrows():
                if not np.isnan(box_row['x']):
                    # Create a Rectangle patch
                    rect = patches.Rectangle(
                        (box_row['x'], box_row['y']),
                        box_row['width'],
                        box_row['height'],
                        linewidth=2,
                        edgecolor='r',
                        facecolor='none'
                    )

                    axes[i].add_patch(rect)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()


#Normal cases (Target = 0)
plot_image_samples(df_class, target=0, num_samples=5)

#Pneumonia cases (Target = 1)
plot_image_samples(df_class, target=1, num_samples=5)

# 5. Data Preprocessing & Splitting

This is the final preparation step before we build our model. We split the master DataFrame (`df_class`) into training and validation sets.

- **Stratification**: We use the `stratify` option to ensure that the proportion of normal-to-pneumonia cases is the same in both the training (`df_train`) and validation (`df_val`) sets. This is a critical best practice for imbalanced datasets as it leads to more reliable model evaluation.

In [ ]:
df_train, df_val = train_test_split(
    df_class,
    test_size=0.2, # Using 20% of the data for validation
    random_state=42,
    stratify=df_class['Target'] # Stratify ensures similar class distribution in both sets
)

print(f"\nData successfully split:")
print(f"Training set size:   {len(df_train)}")
print(f"Validation set size: {len(df_val)}")
print("\nTraining Set Class Distribution:")
print(df_train['Target'].value_counts(normalize=True))
print("\nValidation Set Class Distribution:")
print(df_val['Target'].value_counts(normalize=True))

This helper function takes a dataframe and an ImageDataGenerator instance. It loads a specific image, applies the random transformations defined in the generator, and plots the original image alongside several augmented versions.

In [ ]:
def plot_augmented_samples(df, datagen, target, num_augments=5, figsize=(20, 5)):

    sample_row = df[df['Target'] == str(target)].sample(1).iloc[0]
    img_path = sample_row['filepath']
    patient_id = sample_row['patientId']

    # 2. Load and Prepare the image
    original_img = load_img(img_path, target_size=(224, 224))

    # Shape becomes (1, 224, 224, 3)
    x = img_to_array(original_img)
    x = np.expand_dims(x, axis=0)

    # 3. Setup the Plot
    fig, axes = plt.subplots(1, num_augments + 1, figsize=figsize)
    fig.suptitle(f"Augmentation Effects on Patient: {patient_id[:8]}... (Target: {target})", fontsize=16)

    axes[0].imshow(original_img)
    axes[0].set_title("Original", fontsize=12, fontweight='bold')
    axes[0].axis('off')


    # The .flow() method generates batches indefinitely.
    iterator = datagen.flow(x, batch_size=1)

    for i in range(num_augments):

        batch = next(iterator)

        aug_img = batch[0]


        axes[i + 1].imshow(aug_img)
        axes[i + 1].set_title(f"Augment {i+1}", fontsize=10)
        axes[i + 1].axis('off')

    plt.tight_layout()
    plt.show()

Here we define the ImageDataGenerator with specific hyperparameters. These settings (rotation, zoom, horizontal flip) determine how the images are mathematically distorted to create new training examples.

In [ ]:
train_datagen_viz = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

4. Visualizing Pneumonia Cases (Target = 1)


In [ ]:
print("Visualizing Augmentation on a Pneumonia Case (Target=1)...")

plot_augmented_samples(df_class, train_datagen_viz, target=1, num_augments=5)

5. Visualizing Normal Cases (Target = 0)


In [ ]:
plot_augmented_samples(df_class, train_datagen_viz, target=0, num_augments=5)

# 6. Model Preparation: Data Augmentation & Generators

Before we feed our images to the model, we need to process them. We will use Keras's `ImageDataGenerator` for this.

###  Step 1: Define Image Parameters
We first set the image size and batch size — these control the shape and number of images fed to the model at once.


In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32

### Step 2: Create Data Augmentation Generator for Training
The training generator will apply random transformations (rotation, zoom, flipping, etc.)
to make the model more robust and reduce overfitting.


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255.,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

### Step 3: Create a Simple Rescaling Generator for Validation
Validation data should **not** be augmented — we only rescale pixel values to `[0, 1]`
to evaluate the model on clean, unmodified images.


In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255.)

### Step 4: Create Generators That Flow Data from the DataFrame
Now, we connect the image file paths and labels to the generators.  
The generator will:
- Load each image from the directory
- Resize it to 224×224
- Apply augmentation (for training)
- Yield batches automatically during training


In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    directory=PNG_DIR,
    x_col='filename',
    y_col='Target',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    validate_filenames=False
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=df_val,
    directory=PNG_DIR,
    x_col='filename',
    y_col='Target',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False,
    validate_filenames=False
)


### Step 5: Calculate Class Weights
If one class (e.g., *normal*) has far more samples than the other (e.g., *pneumonia*),
we compute **class weights** so the model pays equal attention to both.


In [ ]:
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weight_dict = dict(enumerate(class_weights))

print(f"\nCalculated Class Weights to handle imbalance: {class_weight_dict}")


## Step 7.1.1: Define and Compile the Basic CNN Model

We start with a simple Convolutional Neural Network (CNN) architecture to classify chest X-ray images as **Normal** or **Pneumonia**.  
This model includes:
- Two convolution + pooling layers for feature extraction  
- A dense layer for learning complex patterns  
- A dropout layer to prevent overfitting  
- A sigmoid output layer for binary classification

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


In [ ]:
EPOCHS = 10
start=time.time()
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    class_weight=class_weight_dict,  # handle imbalance
    verbose=1
)
end=time.time()
elapsed=end-start


## Step 7.1.4: Visualize Training Performance

We plot the training and validation **accuracy** and **loss** over epochs to evaluate model performance.  
These plots help us identify:
- Overfitting (training accuracy much higher than validation)
- Underfitting (both accuracies low)
- Convergence (loss decreasing steadily)
  

In [ ]:
plt.figure(figsize=(12,5))

# Accuracy plot
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title('Accuracy')

# Loss plot
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')

plt.show()


1.
We use DenseNet121(224) as the feature extractor because its dense connections make it highly efficient at capturing the subtle textural patterns found in X-rays. We add a custom classification head with BatchNormalization and Dropout to prevent overfitting.

In [ ]:
def build_densenet_model(input_shape=(224, 224, 3)):
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze the base model
    base_model.trainable = False

    # custom classifier head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)  # High dropout to handle the small dataset size vs model complexity
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)

    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return base_model, model

base_model, model = build_densenet_model()
model.summary()

2. Configure Callbacks
We set up callbacks to ensure we save the best version of the model and adjust the learning rate dynamically if the model gets stuck on a plateau.

In [ ]:
callbacks_list = [
    ModelCheckpoint('best_pneumonia_model.h5', monitor='val_loss', save_best_only=True, mode='min'),

    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1, min_lr=1e-7),

    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1)
]

3. Phase 1: Warm-up Training
In this phase, we only train the custom head layers. This allows the random weights of our new dense layers to align with the pre-trained features of the base model without disrupting them.

In [ ]:
# Use a standard learning rate for the head
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train for a few epochs
history_warmup = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator,
    class_weight=class_weight_dict,
    verbose=1
)

4. Phase 2: Deep Fine-Tuning
Now that the head is stable, we unfreeze the entire model. We recompile with a very low learning rate (1e-5) to gently adjust the pre-trained weights to recognize X-ray specific features (like lung opacity) rather than generic objects.

In [ ]:
# Unfreeze all layers of the base model
base_model.trainable = True

# low learning rate to prevent catastrophic forgetting
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history_finetune = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    class_weight=class_weight_dict,
    callbacks=callbacks_list,
    verbose=1
)

5. Visualize Combined History
This custom plotting function stitches the history from Phase 1 and Phase 2 together, allowing you to see the full training trajectory and the impact of fine-tuning.

In [ ]:
def plot_full_history(h1, h2):
    acc = h1.history['accuracy'] + h2.history['accuracy']
    val_acc = h1.history['val_accuracy'] + h2.history['val_accuracy']
    loss = h1.history['loss'] + h2.history['loss']
    val_loss = h1.history['val_loss'] + h2.history['val_loss']

    plt.figure(figsize=(15, 6))

    # Plot Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(acc, label='Train Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.axvline(x=len(h1.history['accuracy'])-0.5, color='green', linestyle='--', label='Fine-Tuning Start')
    plt.title('Model Accuracy over Phases')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Plot Loss
    plt.subplot(1, 2, 2)
    plt.plot(loss, label='Train Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.axvline(x=len(h1.history['loss'])-0.5, color='green', linestyle='--', label='Fine-Tuning Start')
    plt.title('Model Loss over Phases')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.show()

plot_full_history(history_warmup, history_finetune)

1. Build the Model Architecture
We construct the model using ResNet50 pre-trained on ImageNet. We freeze the base layers so they act as a fixed feature extractor and add a custom classification head.

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze the base model layers
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

2. Phase 1: Feature Extraction
In this phase, we train only the top layers (the "head") that we just added. We use a standard learning rate (1e-4). This allows the random weights of our new layers to converge without disrupting the powerful pre-trained weights in the ResNet base.

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator,
    class_weight=class_weight_dict,
    verbose=1
)

3. Phase 2: Fine-Tuning
Now that the head is trained, we unfreeze the top 20% of the base model. We switch to a much lower learning rate (1e-5) to gently adapt these pre-trained layers to the specific features of Chest X-rays. We also introduce callbacks for early stopping and learning rate reduction.

In [ ]:
base_model.trainable = True

# Unfreeze the top 20% of the layers
fine_tune_at = int(len(base_model.layers) * 0.8)

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompile with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
]

history_fine_tune = model.fit(
    train_generator,
    epochs=20,
    initial_epoch=history.epoch[-1] + 1,
    validation_data=validation_generator,
    class_weight=class_weight_dict,
    callbacks=callbacks,
    verbose=1
)

4. Visualize Training History
This function concatenates the history from both phases and plots them on a single graph, marking the point where fine-tuning began.

In [ ]:
def plot_history(history, history_fine, initial_epochs=5):
    acc = history.history['accuracy'] + history_fine.history['accuracy']
    val_acc = history.history['val_accuracy'] + history_fine.history['val_accuracy']
    loss = history.history['loss'] + history_fine.history['loss']
    val_loss = history.history['val_loss'] + history_fine.history['val_loss']

    plt.figure(figsize=(12, 5))

    # Accuracy Plot
    plt.subplot(1, 2, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.axvline(x=initial_epochs-1, color='green', linestyle='--', label='Start Fine-Tuning')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')

    # Loss Plot
    plt.subplot(1, 2, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.axvline(x=initial_epochs-1, color='green', linestyle='--', label='Start Fine-Tuning')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')

    plt.show()

plot_history(history, history_fine_tune)

1. Configuration Upgrade
We update the global constants. Increasing the image size to 320x320 significantly increases the memory footprint, so we must reduce the batch size (e.g., from 32 to 16) to avoid "Out of Memory" (OOM) errors on the GPU

In [ ]:
IMG_SIZE = 320
BATCH_SIZE = 16

We recreate the ImageDataGenerator objects. We keep the augmentation parameters (rotation, zoom, shifts) active to prevent overfitting, especially now that the model has more pixels (features) to learn from.

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(rescale=1./255)

Finally, we generate the new data flows using the 320x320 target size. These new generator objects (train_generator_320) will be passed to the model during training.

In [ ]:
train_generator_320 = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=PNG_DIR,
    x_col='filename',
    y_col='Target',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

validation_generator_320 = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=PNG_DIR,
    x_col='filename',
    y_col='Target',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

1. Rebuild Model Architecture
We instantiate a new DenseNet121 with the updated input shape. We attach the exact same classifier head structure as before to ensure the capacity remains sufficient for the task.

In [ ]:
base_model = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base_model.trainable = False

# classifier head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)

predictions = Dense(1, activation=None)(x)

final_model_320 = Model(inputs=base_model.input, outputs=predictions)
final_model_320.summary()

2. Phase 1: Adapter Warm-up
Since the DenseNet base weights are for ImageNet (natural images) and our head weights are random, we run a short "warm-up." This aligns the new head with the base features before we unlock the whole network.

In [ ]:
optimizer = SGD(learning_rate=1e-3, momentum=0.9, clipnorm=1.0)

final_model_320.compile(
    optimizer=optimizer,
    loss=BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

history_warmup_320 = final_model_320.fit(
    train_generator_320,
    epochs=3,
    validation_data=validation_generator_320,
    class_weight=class_weight_dict,
    verbose=1
)

3. Phase 2: Deep Fine-Tuning
Now we unfreeze the entire model. Because we are working with high-resolution medical images, we use an ultra-low learning rate. This allows the pre-trained filters to fine-tune their edge and texture detection specifically for pneumonia opacities without destroying the knowledge they already have.

In [ ]:
base_model.trainable = True

optimizer_fine = SGD(learning_rate=1e-5, momentum=0.9, clipnorm=1.0)

final_model_320.compile(
    optimizer=optimizer_fine,
    loss=BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

callbacks_list_320 = [
    ModelCheckpoint('best_320_densenet.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1),

    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7, verbose=1),

    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1)
]

history_fine_320 = final_model_320.fit(
    train_generator_320,
    epochs=25,
    validation_data=validation_generator_320,
    class_weight=class_weight_dict,
    callbacks=callbacks_list_320,
    verbose=1
)

 Build the Ensemble Partner: ResNet50V2
 We use ResNet50V2 for this purpose. By training this model on the same 320x320 data, it will learn different feature representations, increasing the ensemble's overall generalization power.


In [ ]:

IMG_SIZE = 320
BATCH_SIZE = 16

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = Rescaling(scale=2.0, offset=-1.0)(inputs)

base_model = ResNet50V2(weights='imagenet', include_top=False, input_tensor=x)


base_model.trainable = True

# Build Classifier Head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
predictions = Dense(1, activation=None)(x) # Logits output for numerical stability

resnet_model = Model(inputs=inputs, outputs=predictions)

2. Compile and Train
We define the callbacks to save the best weights specifically for this architecture (best_resnet_320.h5) and run the training process using the existing 320x320 data generators.

In [ ]:
resnet_model.compile(
    optimizer=SGD(learning_rate=1e-4, momentum=0.9, clipnorm=1.0),
    loss=BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

callbacks_resnet = [

    ModelCheckpoint('best_resnet_320.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7, verbose=1),
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1)
]

history_resnet = resnet_model.fit(
    train_generator_320,
    epochs=25,
    validation_data=validation_generator_320,
    class_weight=class_weight_dict,
    callbacks=callbacks_resnet,
    verbose=1
)


1. Setup Data & Configuration
We need to ensure the validation data is loaded exactly the same way as during training to guarantee a fair evaluation. We re-create the dataframe split using the same random seed.

In [ ]:
if Path('/kaggle/working/train_png_converted').exists():
    PNG_DIR = Path('/kaggle/working/train_png_converted')
else:
    PNG_DIR = Path('/kaggle/input/rsna-pneu-train-png/orig')


labels_path = Path('/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv')
df_labels = pd.read_csv(labels_path)

df_class = df_labels.drop_duplicates('patientId')[['patientId', 'Target']].copy()
df_class['filename'] = df_class['patientId'].apply(lambda x: f"{x}.png")
df_class['Target'] = df_class['Target'].astype(str)

train_df, val_df = train_test_split(df_class, test_size=0.2, random_state=42, stratify=df_class['Target'])

print(f"Validation set size: {len(val_df)}")

2. Load ResNet Model & Run TTA Prediction
We load the best_resnet_320.h5 model file. Then, we run 5-Pass Test Time Augmentation. This means we predict on the validation set 5 times, applying random augmentations (flips, zooms) each time, and average the results. This mimics how the ensemble will eventually work and gives a better estimate of the model

In [ ]:
resnet_model = load_model('best_resnet_320.h5', compile=False



tta_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10,
    zoom_range=0.1
)


tta_generator = tta_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=PNG_DIR,
    x_col='filename',
    y_col='Target',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)


preds_list = []
for i in range(5):
    print(f"  > Pass {i+1}/5...")
    tta_generator.reset()
    logits = resnet_model.predict(tta_generator, verbose=0)
    preds_list.append(tf.sigmoid(logits).numpy())

resnet_preds = np.mean(preds_list, axis=0)
y_true = tta_generator.classes
y_pred_binary = (resnet_preds > 0.5).astype(int)

Metrics Report & Visualization
Finally, we calculate the standard classification metrics (Accuracy, AUC, Sensitivity, Specificity) and visualize the performance using a Confusion Matrix and ROC Curve.

In [ ]:
acc = accuracy_score(y_true, y_pred_binary)
auc = roc_auc_score(y_true, resnet_preds)
cm = confusion_matrix(y_true, y_pred_binary)
tn, fp, fn, tp = cm.ravel()

sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
precision = tp / (tp + fp)
f1 = 2 * (precision * sensitivity) / (precision + sensitivity)

print(f"\n{'='*40}")
print(f" RESNET50V2 MODEL REPORT")
print(f"{'='*40}")
print(f" Accuracy:       {acc * 100:.2f}%")
print(f"AUC Score:      {auc:.4f}")
print(f"----------------------------------------")
print(f" Sensitivity:    {sensitivity * 100:.2f}%")
print(f"Specificity:    {specificity * 100:.2f}%")
print(f" F1 Score:       {f1:.4f}")
print(f"{'='*40}\n")

plt.figure(figsize=(15, 6))

# Confusion Matrix
plt.subplot(1, 2, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', cbar=False,
            xticklabels=['Normal', 'Pneumonia'],
            yticklabels=['Normal', 'Pneumonia'])
plt.title('ResNet Confusion Matrix', fontsize=14)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

# ROC Curve
fpr, tpr, _ = roc_curve(y_true, resnet_preds)
plt.subplot(1, 2, 2)
plt.plot(fpr, tpr, color='green', lw=3, label=f'ResNet AUC = {auc:.4f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ResNet ROC Curve', fontsize=14)
plt.legend(loc="lower right")

plt.tight_layout()
plt.show()

Load Ensemble Members
We load both trained models. The DenseNet121 serves as our primary expert (higher individual accuracy), while the ResNet50V2 acts as a partner to catch edge cases the DenseNet might miss.

In [ ]:
try:
    model_dense = load_model('best_320_densenet.h5', compile=False)
    print("DenseNet121 (Expert) Loaded.")

    model_resnet = load_model('best_resnet_320.h5', compile=False)
    print(" ResNet50V2 (Partner) Loaded.")

except Exception as e:
    print(f" Error loading models: {e}")

3. Perform Test Time Augmentation (TTA)
Instead of a single prediction, we predict on the validation set 5 times for each model, applying random augmentations (zoom, rotation) each time. We then average these predictions. This statistically reduces the likelihood of a model making a mistake due to image orientation or artifacts.

In [ ]:

tta_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10,
    zoom_range=0.1
)

tta_generator = tta_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=PNG_DIR,
    x_col='filename',
    y_col='Target',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

NUM_TTA = 5
preds_dense_list = []
preds_resnet_list = []

for i in range(NUM_TTA):
    print(f"   > Pass {i+1}/{NUM_TTA}...")
    tta_generator.reset()

    p1 = model_dense.predict(tta_generator, verbose=1)
    preds_dense_list.append(tf.sigmoid(p1).numpy())


    p2 = model_resnet.predict(tta_generator, verbose=1)
    preds_resnet_list.append(tf.sigmoid(p2).numpy())


avg_pred_dense = np.mean(preds_dense_list, axis=0)
avg_pred_resnet = np.mean(preds_resnet_list, axis=0)



 Ensemble Aggregation:
We combine the predictions using a Weighted Average. We assign a weight of 0.6 to the DenseNet (better performance) and 0.4 to the ResNet.

In [ ]:
# Weighted Ensemble
final_ensemble_preds = (avg_pred_dense * 0.6) + (avg_pred_resnet * 0.4)

# Extract Ground Truth labels
y_true = tta_generator.classes

Threshold Optimization & Metrics:
We dynamically scan thresholds from 0.0 to 1.0 to find the specific cutoff that maximizes accuracy. This is often better than using the default 0.5 threshold for imbalanced medical datasets.

In [ ]:
thresholds = np.arange(0.0, 1.0, 0.01)
accuracies = []

# Test every threshold
for t in thresholds:
    y_temp = (final_ensemble_preds > t).astype(int)
    acc = accuracy_score(y_true, y_temp)
    accuracies.append(acc)

best_idx = np.argmax(accuracies)
best_threshold = thresholds[best_idx]
best_acc = accuracies[best_idx]

ensemble_auc = roc_auc_score(y_true, final_ensemble_preds)

print(f"\n{'='*40}")
print(f"{'='*40}")
print(f" Ensemble AUC Score:        {ensemble_auc:.4f}")
print(f" Optimized Accuracy:        {best_acc * 100:.2f}%")
print(f" (at Threshold: {best_threshold:.2f})")
print(f"{'='*40}\n")

6. Visualization
Finally, we plot the ROC Curve (to show diagnostic ability) and the Confusion Matrix (at the optimized threshold) to visualize false positives vs. false negatives.

In [ ]:
plt.figure(figsize=(14, 6))

fpr, tpr, _ = roc_curve(y_true, final_ensemble_preds)
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, color='purple', lw=3, label=f'Ensemble AUC = {ensemble_auc:.4f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Ensemble ROC Curve')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)

y_pred_best = (final_ensemble_preds > best_threshold).astype(int)
cm = confusion_matrix(y_true, y_pred_best)

plt.subplot(1, 2, 2)
sns.heatmap(cm, annot=True, fmt='d', cmap='Purples',
            xticklabels=['Normal', 'Pneumonia'],
            yticklabels=['Normal', 'Pneumonia'])
plt.title(f'Confusion Matrix (Threshold {best_threshold:.2f})')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

plt.tight_layout()
plt.show()

Domain Adaptation: Fine-Tuning on Adult X-rays (NIH)
Our DenseNet model is currently an expert on children's X-rays. To make it robust for adults, we will fine-tune it using the NIH Chest X-ray Dataset. We use a very low learning rate to gently introduce adult anatomical features without forgetting the pneumonia features it has already learned.

In [ ]:
NIH_CSV = None
NIH_IMG_DIR = None
MODEL_PATH = None

for root, dirs, files in os.walk('/kaggle/input'):
    if 'sample_labels.csv' in files:
        NIH_CSV = os.path.join(root, 'sample_labels.csv')
        if 'images' in dirs:
            NIH_IMG_DIR = os.path.join(root, 'images')
        else:
            NIH_IMG_DIR = root


    if 'best_320_densenet.h5' in files:
        MODEL_PATH = os.path.join(root, 'best_320_densenet.h5')

if not NIH_CSV:
    print("\n Could not find")
    raise FileNotFoundError("Missing Dataset")
else:
    print(f"Found Data")

if not MODEL_PATH:
    print("\nCould not find 'best_320_densenet.h5'.")
    if os.path.exists('best_320_densenet.h5'):
        MODEL_PATH = 'best_320_densenet.h5'
        print(f" Found model in working directory: {MODEL_PATH}")
    else:
        raise FileNotFoundError("Missing Model")
else:
    print(f" Found Model: {MODEL_PATH}")

Data Processing
The NIH dataset labels are more complex than the pediatric one. We filter the data to strictly separate "Pneumonia" cases from "No Finding" (Normal) cases, ignoring other diseases for this specific binary classifier.

In [ ]:
df = pd.read_csv(NIH_CSV)

def get_label(labels):
    if 'Pneumonia' in labels: return 'PNEUMONIA'
    if labels == 'No Finding': return 'NORMAL'
    return 'OTHER'

df['class'] = df['Finding Labels'].apply(get_label)
df_adult = df[df['class'] != 'OTHER'].copy()

train_df, val_df = train_test_split(df_adult, test_size=0.2, random_state=42, stratify=df_adult['class'])
print(f"Found {len(df_adult)} valid images. Training on {len(train_df)}.")

Generators & TrainingWe initialize the generators with the same high resolution ($320 \times 320$) used previously. Then, we load the pediatric model and train it for a few epochs on the adult data using SGD with a low learning rate (1e-5) to ensure stable convergence.

In [ ]:
IMG_SIZE = 320
BATCH_SIZE = 16

train_datagen = ImageDataGenerator(
    rescale=1./255, rotation_range=10, zoom_range=0.1, horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df, directory=NIH_IMG_DIR, x_col='Image Index', y_col='class',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='binary'
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df, directory=NIH_IMG_DIR, x_col='Image Index', y_col='class',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='binary', shuffle=False
)


model = load_model(MODEL_PATH, compile=False)

model.compile(
    optimizer=SGD(learning_rate=1e-5, momentum=0.9, clipnorm=1.0),
    loss=BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

callbacks = [
    ModelCheckpoint('best_universal_densenet.h5', monitor='val_loss', save_best_only=True, mode='min'),
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
]

history = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1
)


Locate Dataset & Model :
This block automatically searches the Kaggle directories to find the NIH dataset CSV, image folder, and your trained model file (best_universal_densenet.h5).

In [ ]:
NIH_CSV = None
NIH_IMG_DIR = None
MODEL_PATH = None

for root, dirs, files in os.walk('/kaggle/input'):
    if 'sample_labels.csv' in files:
        NIH_CSV = os.path.join(root, 'sample_labels.csv')
        NIH_IMG_DIR = os.path.join(root, 'images') if 'images' in dirs else root
    if 'best_universal_densenet.h5' in files:
        MODEL_PATH = os.path.join(root, 'best_universal_densenet.h5')

if not MODEL_PATH and os.path.exists('best_universal_densenet.h5'):
    MODEL_PATH = 'best_universal_densenet.h5'

if not NIH_CSV or not MODEL_PATH:
    print(f"\nData: {NIH_CSV}\nModel: {MODEL_PATH}")


Data Preparation:
We filter the NIH dataset to keep only "Pneumonia" and "Normal" (No Finding) cases. Crucially, we perform the exact same split (with random_state=42) used during training to regenerate the Validation Set—data the model has never trained on.

In [ ]:
df = pd.read_csv(NIH_CSV)

def get_label(labels):
    if 'Pneumonia' in labels: return 'PNEUMONIA'
    if labels == 'No Finding': return 'NORMAL'
    return 'OTHER'

df['class'] = df['Finding Labels'].apply(get_label)
df_adult = df[df['class'] != 'OTHER'].copy()

val_df = train_test_split(df_adult, test_size=0.2, random_state=42, stratify=df_adult['class'])

Run Inference :
We create a data generator for the validation set and run the model to get probability predictions

In [ ]:

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=NIH_IMG_DIR,
    x_col='Image Index',
    y_col='class',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

model = load_model(MODEL_PATH, compile=False)

val_generator.reset()
logits = model.predict(val_generator, verbose=1)
probs = tf.sigmoid(logits).numpy()

We evaluate the model using standard metrics: Accuracy, AUC, Sensitivity (Recall), and Specificity.

In [ ]:
y_true = val_generator.classes

y_pred = (probs > 0.5).astype(int)

acc = accuracy_score(y_true, y_pred)
auc = roc_auc_score(y_true, probs)
cm = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = cm.ravel()

sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

print(f"\n{'='*40}")
print(f" UNIVERSAL MODEL RESULTS (ADULTS)")
print(f"{'='*40}")
print(f" Accuracy:       {acc * 100:.2f}%")
print(f"AUC Score:      {auc:.4f}")
print(f"----------------------------------------")
print(f" Sensitivity:    {sensitivity * 100:.2f}% (Caught Pneumonia)")
print(f" Specificity:    {specificity * 100:.2f}% (Cleared Normal)")
print(f"{'='*40}\n")

Finally, we plot the Confusion Matrix to see exact error counts and the ROC Curve to visualize the trade-off between sensitivity and false positives.

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens')
plt.title('Confusion Matrix (Adults)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.xticks([0.5, 1.5], ['Normal', 'Pneumonia'])
plt.yticks([0.5, 1.5], ['Normal', 'Pneumonia'])

plt.subplot(1, 2, 2)
fpr, tpr, _ = roc_curve(y_true, probs)
plt.plot(fpr, tpr, color='green', lw=3, label=f'AUC = {auc:.4f}')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.title('ROC Curve')
plt.legend()
plt.show()

Universal Training on CheXpert

This cell sets up the environment and specifically looks for your previously trained best_universal_densenet.h5. It also defines the paths for the CheXpert dataset, which has a unique directory structure on Kaggle.

In [ ]:
IMG_SIZE = 320
BATCH_SIZE = 16
OLD_MODEL_NAME = 'best_universal_densenet.h5'
NEW_MODEL_NAME = 'best_chexpert_densenet.h5'

DATA_ROOT = Path('/kaggle/input/chexpert')
CSV_PATH = DATA_ROOT / 'train.csv'

MODEL_PATH = None
possible_paths = [
    Path(f'/kaggle/input/my-pneumonia-model/{OLD_MODEL_NAME}'),
    Path(OLD_MODEL_NAME),
    Path(f'/kaggle/working/{OLD_MODEL_NAME}')
]

for p in possible_paths:
    if p.exists():
        MODEL_PATH = p
        print(f"Found previous model at: {MODEL_PATH}")
        break

if not MODEL_PATH:
    print(f" Could not find old model.)

Data Cleaning & Path Fixing:
The CheXpert CSV file contains relative paths (e.g., CheXpert-v1.0-small/train/...) that do not match the Kaggle input directory structure. This cell fixes those paths and filters the dataset to only include Pneumonia (Positive) and No Finding (Normal) cases.

In [ ]:
df = pd.read_csv(CSV_PATH)

def get_label(row):
    if row['Pneumonia'] == 1.0: return 'PNEUMONIA'
    if row['No Finding'] == 1.0: return 'NORMAL'
    return 'OTHER'

df['class'] = df.apply(get_label, axis=1)
df_clean = df[df['class'] != 'OTHER'].copy()

def fix_path_exact(p):
    if 'train/' in p:
        idx = p.find('train/')
        relative_path = p[idx:] #
        return str(DATA_ROOT / relative_path)
    return str(p)

df_clean['path_fixed'] = df_clean['Path'].apply(fix_path_exact)

if os.path.exists(df_clean['path_fixed'].iloc[0]):
    print(f" Path check passed! Example: {df_clean['path_fixed'].iloc[0]}")
else:
    print(f" Path check FAILED: {df_clean['path_fixed'].iloc[0]}")

if len(df_clean) > 50000:
    df_clean = df_clean.sample(50000, random_state=42)

train_df, val_df = train_test_split(df_clean, test_size=0.1, random_state=42, stratify=df_clean['class'])
print(f"Training on {len(train_df)} images")

Create Generators :
We create the data generators using the fixed paths. Note that we use validate_filenames=False to speed up initialization, since we manually verified the paths in the previous step.

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='path_fixed',
    y_col='class',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    validate_filenames=False
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='path_fixed',
    y_col='class',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False,
    validate_filenames=False
)

Fine-Tune on CheXpert :
We load the previous best model and train it on the new data. We use a very low learning rate (1e-5) to ensure the model adapts to the new data source (CheXpert) without forgetting what it learned from the previous datasets (NIH and Pediatric).

In [ ]:
if MODEL_PATH:
    print(f"\n📥 Loading {OLD_MODEL_NAME}...")
    model = load_model(MODEL_PATH, compile=False)
else:

    from tensorflow.keras.applications import DenseNet121
    from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
    base = DenseNet121(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    pred = Dense(1, activation=None)(x)
    model = Model(inputs=base.input, outputs=pred)

model.compile(
    optimizer=SGD(learning_rate=1e-5, momentum=0.9, clipnorm=1.0),
    loss=BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

callbacks = [
    ModelCheckpoint(NEW_MODEL_NAME, monitor='val_loss', save_best_only=True, mode='min'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1
)

1. Threshold Optimization
Instead of using a default threshold of 0.5, we loop through values from 0.0 to 1.0 to find the specific cutoff that maximizes the model's accuracy.

In [ ]:
y_true = test_generator.classes

print("🔍 Finding Best Threshold...")
thresholds = np.arange(0.0, 1.0, 0.01)
best_acc = 0
best_thresh = 0.5

for t in thresholds:
    temp_preds = (probs > t).astype(int)

    acc = accuracy_score(y_true, temp_preds)

    if acc > best_acc:
        best_acc = acc
        best_thresh = t

print(f" Best Accuracy: {best_acc:.4f} at Threshold: {best_thresh:.2f}")

2. Compute Clinical Metrics
Once the best threshold is found, we generate the final binary predictions. We then use the Confusion Matrix to calculate Sensitivity (how well we detect Pneumonia) and Specificity (how well we ignore Normal cases), along with the AUC score.

In [ ]:
y_pred_final = (probs > best_thresh).astype(int)

final_auc = roc_auc_score(y_true, probs)

cm = confusion_matrix(y_true, y_pred_final)
tn, fp, fn, tp = cm.ravel()


sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
precision = tp / (tp + fp)

print(f"\n{'='*40}")
print(f" FINAL METRICS REPORT")
print(f"{'='*40}")
print(f" AUC Score:       {final_auc:.4f}")
print(f"----------------------------------------")
print(f" Sensitivity:     {sensitivity * 100:.2f}% (Recall)")
print(f" Specificity:     {specificity * 100:.2f}% (True Neg Rate)")
print(f" Precision:       {precision * 100:.2f}%")
print(f"{'='*40}\n")

3. Visualize Confusion Matrix
This cell visualizes the results using the specific threshold derived above.

In [ ]:
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Purples',
            xticklabels=['Normal', 'Pneumonia'],
            yticklabels=['Normal', 'Pneumonia'])

plt.title(f'Confusion Matrix (Threshold {best_thresh:.2f})')
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.show()

Detailed Metrics Report:
This cell takes the predictions generated in the previous step and computes the performance statistics. Note that we manually calculate Specificity (the ability to correctly identify normal cases), as this is just as important as detecting pneumonia in a clinical setting.

In [ ]:
y_pred_binary = (probs > THRESHOLD).astype(int)
y_true_binary = y_true

accuracy = accuracy_score(y_true_binary, y_pred_binary)
sensitivity = recall_score(y_true_binary, y_pred_binary)
precision = precision_score(y_true_binary, y_pred_binary)
f1 = f1_score(y_true_binary, y_pred_binary)

tn, fp, fn, tp = confusion_matrix(y_true_binary, y_pred_binary).ravel()
specificity = tn / (tn + fp)


print(f"\n{'='*40}")
print(f"🩺  FINAL MODEL PERFORMANCE REPORT")
print(f"{'='*40}")
print(f"Model:           DenseNet121 (Universal)")
print(f"Threshold:       {THRESHOLD}")
print(f"Test Set Size:   {len(y_true)} images")
print(f"{'-'*40}")
print(f" Accuracy:      {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"  F1-Score:      {f1:.4f}")
print(f" AUC Score:     {roc_auc:.4f}")
print(f"{'-'*40}")
print(f" Sensitivity:   {sensitivity:.4f}  (Ability to detect Pneumonia)")
print(f" Specificity:   {specificity:.4f}  (Ability to identify Normal)")
print(f" Precision:     {precision:.4f}  (Trustworthiness of a positive prediction)")
print(f"{'='*40}\n")

print("Detailed Classification Report:")
print(classification_report(y_true_binary, y_pred_binary, target_names=['Normal', 'Pneumonia']))

1.Configuration & ImportsWe set up the environment and define thebest ($0.15$) that balances sensitivity and specificity for this  model.

In [ ]:
IMG_SIZE = 320
BATCH_SIZE = 16
THRESHOLD = 0.15
MODEL_FILE = 'model.h5'

2. Locate Model & Data
This block ensures we can find the model file (whether it's from the current session or an uploaded dataset) and sets up the paths for the test data.

In [ ]:

possible_paths = [
    Path(f'/kaggle/working/{MODEL_FILE}'),
    Path(f'/kaggle/input/my-pneumonia-model/{MODEL_FILE}'),
    Path(MODEL_FILE)
]

MODEL_PATH = None
for p in possible_paths:
    if p.exists():
        MODEL_PATH = p
        break

if not MODEL_PATH:
    print(f" Error: Could not find '{MODEL_FILE}'")
    raise FileNotFoundError("Model file missing.")

print(f" Found Model at: {MODEL_PATH}")

labels_path = Path('/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv')

if Path('/kaggle/working/train_png_converted').exists():
    PNG_DIR = Path('/kaggle/working/train_png_converted')
else:
    PNG_DIR = Path('/kaggle/input/rsna-pneu-train-png/orig')

if not labels_path.exists():
    raise FileNotFoundError(" RSNA Dataset not found.")

3. Prepare Test Data
We reconstruct the data split to isolate the Validation Set (20%). This ensures we are evaluating the model on data it has never seen during training.

In [ ]:

df_labels = pd.read_csv(labels_path)
df_class = df_labels.drop_duplicates('patientId')[['patientId', 'Target']].copy()
df_class['filename'] = df_class['patientId'].apply(lambda x: f"{x}.png")
df_class['Target'] = df_class['Target'].astype(str)

val_df = train_test_split(df_class, test_size=0.2, random_state=42, stratify=df_class['Target'])

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=PNG_DIR,
    x_col='filename',
    y_col='Target',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

4. Load Model & Predict
We load the trained DenseNet model and generate probability predictions for every image in the validation set.

In [ ]:
model = load_model(MODEL_PATH, compile=False)

logits = model.predict(val_generator, verbose=1)
probs = tf.sigmoid(logits).numpy()
y_true = val_generator.classes



5. Generate Metrics & Charts
This final cell calculates the AUC score and generates two professional-grade plots:

Confusion Matrix: Shows exactly how many Pneumonia cases were caught vs. missed at the chosen threshold.

ROC Curve: Visualizes the model's diagnostic ability across all possible thresholds.

In [ ]:

fpr, tpr, _ = roc_curve(y_true, probs)
roc_auc = auc(fpr, tpr)

y_pred = (probs > THRESHOLD).astype(int)
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', annot_kws={"size": 16})
plt.title(f'Confusion Matrix (Threshold {THRESHOLD})', fontsize=14)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks([0.5, 1.5], ['Normal', 'Pneumonia'])
plt.yticks([0.5, 1.5], ['Normal', 'Pneumonia'])

plt.subplot(1, 2, 2)
plt.plot(fpr, tpr, color='darkorange', lw=3, label=f'AUC = {roc_auc:.4f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve', fontsize=14)
plt.legend(loc="lower right")
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()